# MPIN Strength Checker

This notebook contains a complete solution for the MPIN task. It is structured into the following sections:
1.  **Imports and Constants**: Defines the lists of commonly used PINs.
2.  **Core Logic**: Contains the main function to analyze MPIN strength.
3.  **Unit Tests**: A complete test suite to verify the logic.
4.  **Runner Functions**: Functions to execute the tests or start an interactive checker.
5.  **Execution**: Cells to actually run the tests or the interactive checker.

### 1. Imports and Constants

In [1]:
import datetime
import unittest
from typing import List, Dict, Set, Optional, Any

# A set of commonly used or weak 4-digit PINs
COMMONLY_USED_4_DIGIT_PINS: Set[str] = {
    "1234", "9876", "1111", "2222", "3333", "4444", "5555", "6666", "7777", "8888", "9999", "0000",
    "1122", "1212", "2580", "1990", "1991", "1992", "1993", "1994", "1995", "1996", "1997", "1998", "1999",
    "2000", "2001", "2002", "2003", "2004", "2005",
}

# --- Part D: Enhance for a 6-digit PIN ---
# A set of commonly used or weak 6-digit PINs
COMMONLY_USED_6_DIGIT_PINS: Set[str] = {
    "123456", "987654", "111111", "222222", "333333", "444444", "555555", "666666", "777777", "888888", "999999", "000000",
    "123123", "112233",
}

### 2. Core Logic Functions

In [2]:
def _generate_demographic_pins(date_obj: datetime.date, length: int) -> Set[str]:
    """Generates a set of potential PINs from a given date object."""
    if not date_obj:
        return set()
    day, month = f"{date_obj.day:02d}", f"{date_obj.month:02d}"
    year_yy, year_yyyy = f"{date_obj.year % 100:02d}", f"{date_obj.year:04d}"
    pins = set()
    if length == 4:
        pins.update({
            day + month, month + day, year_yyyy, day + year_yy, year_yy + day,
            month + year_yy, year_yy + month, day * 2, month * 2, year_yy * 2
        })
    elif length == 6:
        pins.update({
            day + month + year_yy, month + day + year_yy,
            year_yy + month + day, year_yy + day + month
        })
    return pins

def check_mpin_strength(mpin: str, demographics: Optional[Dict[str, datetime.date]] = None) -> Dict[str, Any]:
    """Analyzes the strength of a 4 or 6-digit MPIN."""
    # --- Part D: Handle both 4 and 6-digit PINs ---
    if not mpin.isdigit() or len(mpin) not in [4, 6]:
        raise ValueError("MPIN must be a 4 or 6-digit string.")
    
    reasons: Set[str] = set()
    pin_length = len(mpin)

    # --- Part A: Check if the MPIN is commonly used ---
    common_pins = COMMONLY_USED_4_DIGIT_PINS if pin_length == 4 else COMMONLY_USED_6_DIGIT_PINS
    if mpin in common_pins:
        reasons.add("COMMONLY_USED")

    # --- Part B & C: Check against demographics and add reasons ---
    if demographics:
        demographic_map = {
            "dob_self": "DEMOGRAPHIC_DOB_SELF",
            "dob_spouse": "DEMOGRAPHIC_DOB_SPOUSE",
            "anniversary": "DEMOGRAPHIC_ANNIVERSARY",
        }
        for key, reason_code in demographic_map.items():
            if demographics.get(key):
                demographic_pins = _generate_demographic_pins(demographics[key], pin_length)
                if mpin in demographic_pins:
                    reasons.add(reason_code)

    # --- Part B & C: Determine final strength and reasons ---
    strength = "WEAK" if reasons else "STRONG"
    return {"strength": strength, "reasons": sorted(list(reasons))}

### 3. Unit Test Class (Part E)

In [3]:
# --- Part E: Write code that tests the above written code ---
class TestMpinChecker(unittest.TestCase):
    @classmethod
    def setUpClass(cls):
        cls.demographics = {
            "dob_self": datetime.date(1992, 8, 15),
            "dob_spouse": datetime.date(1994, 3, 3),
            "anniversary": datetime.date(2018, 11, 20),
        }
    
    def test_strong_4_digit_pin(self):
        self.assertEqual(check_mpin_strength("5839", self.demographics)["strength"], "STRONG")
    def test_weak_4_digit_repeating(self):
        self.assertCountEqual(check_mpin_strength("1111", self.demographics)["reasons"], ["COMMONLY_USED", "DEMOGRAPHIC_ANNIVERSARY"])
    def test_weak_4_digit_sequential(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("1234", self.demographics)["reasons"])
    def test_weak_4_digit_from_self_dob_ddmm(self):
        self.assertIn("DEMOGRAPHIC_DOB_SELF", check_mpin_strength("1508", self.demographics)["reasons"])
    def test_weak_4_digit_from_self_dob_yydd(self):
        self.assertIn("DEMOGRAPHIC_DOB_SELF", check_mpin_strength("9215", self.demographics)["reasons"])
    def test_weak_4_digit_from_self_dob_yyyy(self):
        self.assertCountEqual(check_mpin_strength("1992", self.demographics)["reasons"], ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SELF"])
    def test_weak_4_digit_from_spouse_dob_mmdd(self):
        self.assertIn("DEMOGRAPHIC_DOB_SPOUSE", check_mpin_strength("0303", self.demographics)["reasons"])
    def test_weak_4_digit_from_spouse_dob_yyyy(self):
        self.assertCountEqual(check_mpin_strength("1994", self.demographics)["reasons"], ["COMMONLY_USED", "DEMOGRAPHIC_DOB_SPOUSE"])
    def test_weak_4_digit_from_anniversary_ddmm(self):
        self.assertIn("DEMOGRAPHIC_ANNIVERSARY", check_mpin_strength("2011", self.demographics)["reasons"])
    def test_weak_4_digit_from_anniversary_yymm(self):
        self.assertIn("DEMOGRAPHIC_ANNIVERSARY", check_mpin_strength("1811", self.demographics)["reasons"])
    def test_weak_4_digit_common_pair(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("1122", self.demographics)["reasons"])
    def test_weak_4_digit_keypad_pattern(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("2580", self.demographics)["reasons"])
    def test_strong_6_digit_pin(self):
        self.assertEqual(check_mpin_strength("742198", self.demographics)["strength"], "STRONG")
    def test_weak_6_digit_repeating(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("888888", self.demographics)["reasons"])
    def test_weak_6_digit_sequential(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("987654", self.demographics)["reasons"])
    def test_weak_6_digit_repeating_pattern(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("123123", self.demographics)["reasons"])
    def test_weak_6_digit_from_self_dob_ddmmyy(self):
        self.assertIn("DEMOGRAPHIC_DOB_SELF", check_mpin_strength("150892", self.demographics)["reasons"])
    def test_weak_6_digit_from_self_dob_yymmdd(self):
        self.assertIn("DEMOGRAPHIC_DOB_SELF", check_mpin_strength("920815", self.demographics)["reasons"])
    def test_weak_6_digit_from_spouse_dob_mmddyy(self):
        self.assertIn("DEMOGRAPHIC_DOB_SPOUSE", check_mpin_strength("030394", self.demographics)["reasons"])
    def test_weak_6_digit_from_anniversary_ddmmyy(self):
        self.assertIn("DEMOGRAPHIC_ANNIVERSARY", check_mpin_strength("201118", self.demographics)["reasons"])
    def test_no_demographics_provided_strong(self):
        self.assertEqual(check_mpin_strength("1508")["strength"], "STRONG")
    def test_no_demographics_provided_weak_common(self):
        self.assertIn("COMMONLY_USED", check_mpin_strength("1111")["reasons"])
    def test_invalid_pin_length(self):
        with self.assertRaises(ValueError): check_mpin_strength("123")
        with self.assertRaises(ValueError): check_mpin_strength("12345")
        with self.assertRaises(ValueError): check_mpin_strength("abcdef")

### 4. Runner Functions

In [4]:
def run_all_tests():
    """Runs the full unittest suite."""
    print("--- Running Unit Test Suite ---")
    suite = unittest.TestSuite()
    loader = unittest.TestLoader()
    suite.addTest(loader.loadTestsFromTestCase(TestMpinChecker))
    runner = unittest.TextTestRunner(verbosity=2)
    result = runner.run(suite)
    if result.wasSuccessful():
        print("\n✅ All tests passed successfully!")
    else:
        print("\n❌ Some tests failed.")

def run_interactive_checker():
    """Runs an interactive loop in the console to check custom MPINs."""
    def get_date_from_user(prompt: str) -> Optional[datetime.date]:
        while True:
            date_str = input(prompt)
            if not date_str: return None
            try:
                return datetime.datetime.strptime(date_str, "%Y-%m-%d").date()
            except ValueError:
                print("Invalid format. Please use YYYY-MM-DD or press Enter to skip.")

    print("--- Interactive MPIN Strength Checker ---")
    print("Enter 'quit' at any time to exit.")
    
    while True:
        mpin = input("\nEnter the 4 or 6-digit MPIN to check: ").strip()
        if mpin.lower() == 'quit': break
        
        if not mpin.isdigit() or len(mpin) not in [4, 6]:
            print("Invalid input. Please enter a 4 or 6-digit number.")
            continue
            
        print("\nPlease provide dates (or press Enter to skip).")
        user_demographics = {
            "dob_self": get_date_from_user("Your DOB (YYYY-MM-DD): "),
            "dob_spouse": get_date_from_user("Spouse's DOB (YYYY-MM-DD): "),
            "anniversary": get_date_from_user("Anniversary (YYYY-MM-DD): "),
        }
        
        result = check_mpin_strength(mpin, user_demographics)
        print("\n--- Analysis Result ---")
        print(f"  PIN: {mpin}, Strength: {result['strength']}")
        if result['reasons']:
            print(f"  Reasons: {', '.join(result['reasons'])}")
        print("-----------------------")

### 5. Execute Tests

Run the following cell to execute all the unit tests.

In [5]:
run_all_tests()

test_invalid_pin_length (__main__.TestMpinChecker.test_invalid_pin_length) ... ok
test_no_demographics_provided_strong (__main__.TestMpinChecker.test_no_demographics_provided_strong) ... ok
test_no_demographics_provided_weak_common (__main__.TestMpinChecker.test_no_demographics_provided_weak_common) ... ok
test_strong_4_digit_pin (__main__.TestMpinChecker.test_strong_4_digit_pin) ... ok
test_strong_6_digit_pin (__main__.TestMpinChecker.test_strong_6_digit_pin) ... ok
test_weak_4_digit_common_pair (__main__.TestMpinChecker.test_weak_4_digit_common_pair) ... ok
test_weak_4_digit_from_anniversary_ddmm (__main__.TestMpinChecker.test_weak_4_digit_from_anniversary_ddmm) ... ok
test_weak_4_digit_from_anniversary_yymm (__main__.TestMpinChecker.test_weak_4_digit_from_anniversary_yymm) ... ok
test_weak_4_digit_from_self_dob_ddmm (__main__.TestMpinChecker.test_weak_4_digit_from_self_dob_ddmm) ... ok
test_weak_4_digit_from_self_dob_yydd (__main__.TestMpinChecker.test_weak_4_digit_from_self_dob_yyd

--- Running Unit Test Suite ---


ok
test_weak_4_digit_repeating (__main__.TestMpinChecker.test_weak_4_digit_repeating) ... ok
test_weak_4_digit_sequential (__main__.TestMpinChecker.test_weak_4_digit_sequential) ... ok
test_weak_6_digit_from_anniversary_ddmmyy (__main__.TestMpinChecker.test_weak_6_digit_from_anniversary_ddmmyy) ... ok
test_weak_6_digit_from_self_dob_ddmmyy (__main__.TestMpinChecker.test_weak_6_digit_from_self_dob_ddmmyy) ... ok
test_weak_6_digit_from_self_dob_yymmdd (__main__.TestMpinChecker.test_weak_6_digit_from_self_dob_yymmdd) ... ok
test_weak_6_digit_from_spouse_dob_mmddyy (__main__.TestMpinChecker.test_weak_6_digit_from_spouse_dob_mmddyy) ... ok
test_weak_6_digit_repeating (__main__.TestMpinChecker.test_weak_6_digit_repeating) ... ok
test_weak_6_digit_repeating_pattern (__main__.TestMpinChecker.test_weak_6_digit_repeating_pattern) ... ok
test_weak_6_digit_sequential (__main__.TestMpinChecker.test_weak_6_digit_sequential) ... ok

--------------------------------------------------------------------


✅ All tests passed successfully!


### 6. Run Interactive Checker

Run the following cell to start the interactive checker and test custom inputs.

In [6]:
run_interactive_checker()

--- Interactive MPIN Strength Checker ---
Enter 'quit' at any time to exit.
